<a href="https://colab.research.google.com/github/i-SanMartin/FastAI/blob/main/04_MNIST_Basics/Network_Exercise_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook fastai==2.2.5
import fastbook

     |████████████████████████████████| 727kB 6.2MB/s 
     |████████████████████████████████| 194kB 10.3MB/s 
     |████████████████████████████████| 1.2MB 11.4MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 12.8MB 289kB/s 
     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 776.8MB 20kB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.


In [2]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
path.ls()

(#2) [Path('/root/.fastai/data/mnist_png/training'),Path('/root/.fastai/data/mnist_png/testing')]

In [5]:
Path.BASE_PATH = path

In [6]:
(path/'training').ls()

(#10) [Path('training/2'),Path('training/4'),Path('training/9'),Path('training/1'),Path('training/6'),Path('training/7'),Path('training/5'),Path('training/8'),Path('training/0'),Path('training/3')]

In [7]:
categories = [x for x in range(0,10)]
categories

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [83]:
train_x = torch.tensor([])
train_y = torch.tensor([])
valid_x = torch.tensor([])
valid_y = torch.tensor([])

In [84]:
for x in categories:
  number_imgs = (path/'training'/str(x)).ls().sorted()
  zeros_tensor =  [tensor(Image.open(img)) for img in number_imgs]
  stacked_zeros = torch.stack(zeros_tensor)
  train_x = torch.cat([train_x,stacked_zeros])
  y = torch.tensor([x]*len(number_imgs))
  train_y = torch.cat([train_y,y])

In [85]:
for x in categories:
  number_imgs = (path/'testing'/str(x)).ls().sorted()
  zeros_tensor =  [tensor(Image.open(img)) for img in number_imgs]
  stacked_zeros = torch.stack(zeros_tensor)
  valid_x = torch.cat([valid_x,stacked_zeros])
  y = torch.tensor([x]*len(number_imgs))
  valid_y = torch.cat([valid_y,y])

In [86]:
train_x = train_x.view(-1,28*28).float()/255
valid_x = valid_x.view(-1,28*28).float()/255
train_y = train_y.unsqueeze(1).long()
valid_y = valid_y.unsqueeze(1).long()
train_x.shape, train_y.shape, valid_x.shape, valid_y.shape

(torch.Size([60000, 784]),
 torch.Size([60000, 1]),
 torch.Size([10000, 784]),
 torch.Size([10000, 1]))

In [87]:
dset = list(zip(train_x,train_y))
dl = DataLoader(dset, batch_size=256)
valid_dset = list(zip(valid_x,valid_y))
valid_dl = DataLoader(dset, batch_size=256)
dls = DataLoaders(dl, valid_dl)

In [113]:
simple_net = nn.Sequential(
    nn.Linear(28*28,80),
    nn.ReLU(),
    nn.Linear(80,30),
    nn.ReLU(),
    nn.Linear(30,10),
    nn.Sigmoid()
)

In [105]:
def class_tensor_loss(size, elements, label):
  zeros = torch.zeros((size, elements))
  label = label.long()
  for _z, _l in zip(zeros, label):
    _z[_l] = 1
  return zeros

In [106]:
def loss_f(pred, label):
  loss_tensor = class_tensor_loss(pred.shape[0],pred.shape[1],label)
  return torch.sqrt(torch.square(pred-loss_tensor)).mean()

In [107]:
def calc_grad(model,x,y):
  pred = model(x)
  loss = loss_f(pred,y)
  loss.backward()

In [108]:
class optimizer():
  def __init__(self, parameters, lr):
    self.parameters = list(parameters)
    self.lr = lr
  def step(self):
    for p in self.parameters:
      p.data -= p.grad
  def grads_zero(self):
    for p in self.parameters:
      p.grad = None


In [109]:
opt = optimizer(simple_net.parameters(), 100)

In [110]:
def batch_accuracy(x,y):
    pred = simple_net(train_x[0])
    correct = ((pred == pred.max()).nonzero(as_tuple=True)[0] == y).count()

In [111]:
for i in range(0,10):
  for dx, dy in dl:
    calc_grad(simple_net,dx,dy)
    opt.step()
    opt.grads_zero()
  print(loss)

tensor(0.1000, grad_fn=<MeanBackward0>)
tensor(0.1000, grad_fn=<MeanBackward0>)
tensor(0.1000, grad_fn=<MeanBackward0>)
tensor(0.1000, grad_fn=<MeanBackward0>)
tensor(0.1000, grad_fn=<MeanBackward0>)
tensor(0.1000, grad_fn=<MeanBackward0>)
tensor(0.1000, grad_fn=<MeanBackward0>)
tensor(0.1000, grad_fn=<MeanBackward0>)
tensor(0.1000, grad_fn=<MeanBackward0>)
tensor(0.1000, grad_fn=<MeanBackward0>)


In [114]:
pr = simple_net(train_x[0])
pr

tensor([0.5294, 0.4736, 0.5216, 0.5296, 0.4854, 0.4935, 0.5068, 0.5332, 0.5508, 0.4882], grad_fn=<SigmoidBackward>)